In [10]:
import pandas as pd
import numpy as np
from datetime import datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX

# =============================
# 1. LOAD DATA
# =============================

df = pd.read_csv('./Task2/Nat_Gas.csv')
df['Dates'] = pd.to_datetime(df['Dates'])
df.set_index('Dates', inplace=True)
df = df.asfreq('M')
df['Prices'] = df['Prices'].astype(float)

C:\Users\athil\AppData\Local\Temp\ipykernel_23460\1686011175.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Dates'] = pd.to_datetime(df['Dates'])
C:\Users\athil\AppData\Local\Temp\ipykernel_23460\1686011175.py:13: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df.asfreq('M')


In [11]:
# =============================
# 2. ADD SEASONALITY (Fourier terms)
# =============================

def add_fourier_terms(df, period=12, K=3):
    t = np.arange(len(df))
    for k in range(1, K+1):
        df[f'sin_{k}'] = np.sin(2 * np.pi * k * t / period)
        df[f'cos_{k}'] = np.cos(2 * np.pi * k * t / period)
    return df

df = add_fourier_terms(df)
exog = df.drop(columns=['Prices'])

In [12]:
# =============================
# 3. FIT MODEL
# =============================

model = SARIMAX(
    df['Prices'],
    order=(1,0,0),
    seasonal_order=(0,0,0,0),
    exog=exog,
    enforce_stationarity=False,
    enforce_invertibility=False
)

results = model.fit()
print(results.summary())

                               SARIMAX Results                                
Dep. Variable:                 Prices   No. Observations:                   48
Model:               SARIMAX(1, 0, 0)   Log Likelihood                  -4.661
Date:                Tue, 02 Dec 2025   AIC                             25.322
Time:                        18:41:40   BIC                             40.123
Sample:                    10-31-2020   HQIC                            30.891
                         - 09-30-2024                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sin_1          0.6790      0.106      6.436      0.000       0.472       0.886
cos_1         -0.0420      0.116     -0.360      0.719      -0.270       0.186
sin_2         -0.0510      0.060     -0.846      0.3

c:\Users\athil\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [13]:
# =============================
# 4. FUNCTION: Pricing 
# ==============================
def price_gas_storage_contract(injection_dates,
                               withdrawal_dates,
                               estimate_price_func,
                               rate_per_period,
                               max_storage_volume,
                               storage_cost_per_month,
                               injection_withdrawal_cost,
                               transport_cost_per_trip):

    current_volume = 0
    total_value = 0

    # Combine & sort all actions
    actions = []

    for d in injection_dates:
        actions.append((pd.to_datetime(d), 'inject'))

    for d in withdrawal_dates:
        actions.append((pd.to_datetime(d), 'withdraw'))

    actions = sorted(actions, key=lambda x: x[0])

    # Duration for storage cost
    start = min(pd.to_datetime(injection_dates))
    end = max(pd.to_datetime(withdrawal_dates))
    months = (end.to_period('M') - start.to_period('M')).n + 1

    # STORAGE COST
    total_storage_cost = months * storage_cost_per_month
    total_value -= total_storage_cost


    # PROCESS EACH ACTION
    for action_date, action in actions:

        price = estimate_price_func(str(action_date.date()))

        if action == 'inject':

            possible_injection = min(rate_per_period, max_storage_volume - current_volume)

            if possible_injection <= 0:
                continue

            injection_cost = possible_injection * price
            total_value -= injection_cost
            current_volume += possible_injection

            # Operational cost
            total_value -= injection_withdrawal_cost
            total_value -= transport_cost_per_trip

        elif action == 'withdraw':

            possible_withdrawal = min(rate_per_period, current_volume)

            if possible_withdrawal <= 0:
                continue

            sale_revenue = possible_withdrawal * price
            total_value += sale_revenue
            current_volume -= possible_withdrawal

            # Operational cost
            total_value -= injection_withdrawal_cost
            total_value -= transport_cost_per_trip

    return round(total_value, 2)


In [14]:
# =============================
# 5. FUNCTION: PREDICT PRICE FOR ANY DATE
# =============================
def estimate_price(query_date: str):
    qdate = pd.to_datetime(query_date)

    # If in historical data → interpolate
    if qdate <= df.index.max():
        idx = np.searchsorted(df.index.values, np.datetime64(qdate))
        if idx == 0:
            return float(df.iloc[0]['Prices'])

        before = df.iloc[idx - 1]
        after = df.iloc[idx] if idx < len(df) else before

        total_days = (after.name - before.name).days
        passed_days = (qdate - before.name).days

        return float(
            before['Prices'] + 
            (after['Prices'] - before['Prices']) 
            * (passed_days / total_days)
        )

    # If future → forecast
    periods = (qdate.to_period('M') - df.index[-1].to_period('M')).n
    future_index = pd.date_range(df.index[-1] + pd.offsets.MonthEnd(),
                                 periods=periods,
                                 freq='M')

    future_df = pd.DataFrame(index=future_index)
    future_df = add_fourier_terms(future_df)

    forecast = results.get_forecast(
        steps=periods,
        exog=future_df
    )

    value = forecast.predicted_mean.iloc[-1]
    return float(value)


In [15]:
value = price_gas_storage_contract(
    injection_dates=[
        "2024-05-31",
        "2024-06-30"
    ],

    withdrawal_dates=[
        "2024-11-30",
        "2024-12-31"
    ],

    estimate_price_func=estimate_price,  # Your function

    rate_per_period=800000,          # 800k MMBtu per action
    max_storage_volume=1000000,      # 1M MMBtu capacity

    storage_cost_per_month=100000,   # $100k per month
    injection_withdrawal_cost=10000, # per action
    transport_cost_per_trip=60000    # per trip
)

print("Contract Value: $", value)


C:\Users\athil\AppData\Local\Temp\ipykernel_23460\2386263759.py:27: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  future_index = pd.date_range(df.index[-1] + pd.offsets.MonthEnd(),
C:\Users\athil\AppData\Local\Temp\ipykernel_23460\2386263759.py:27: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  future_index = pd.date_range(df.index[-1] + pd.offsets.MonthEnd(),


Contract Value: $ 251217.07
